In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 27 16:44:23 2018

@author: dell
""" 

from collections import defaultdict
import jieba
jieba.load_userdict("../data/userdict.txt")  #手动添加的用于分词的词汇
import jieba.posseg as pseg 

 
#读取txt文件返回行列表
def readLines(file):
    res=[]
    f=open(file,'r', encoding='UTF-8')
    lines=f.readlines()
    for line in lines:
        line=line.strip()
        res.append(line)
    return res



"""
1. 文本切割
将句子切割为单词，去除停用词
"""
 
def sent2word(sentence):
    segList = jieba.cut(sentence)
    segResult = []
    for w in segList:
        segResult.append(w)
 
    stopwords = readLines('../data/stop_words.txt') #停用词表
    newSent = []
    for word in segResult:
        if word in stopwords:
            continue
        else:
            newSent.append(word)
    return newSent  # list


"""
2. 情感定位
"""
def classifyWords(sen):   #sen ：map（词汇，位置）
    # (1) 情感词
    senList = readLines('../data/sentiment.txt')
    senDict = defaultdict()  #map
    for s in senList:
        s = s.split()
        senDict[s[0]] = s[1]    #  s[0]是词汇，s[1]是分数，


        
    # (2) 否定词
    notList = readLines('../data/notDict.txt')

    
    senWord = defaultdict()  # map
    notWord = defaultdict()  #map
    #degreeWord = defaultdict()
    
    for word in sen.keys():
        if word in senDict.keys() and word not in notList:
            senWord[sen[word]] = senDict[word] # 情感词分数，sen[word]是位置，senDict[word]是分数
        elif word in notList:
            notWord[sen[word]] = -1  #否定词  -1分
    return senWord, notWord #  


"""
3. 情感聚合
"""
def scoreSent(senWord, notWord, segResult):
    W = 1
    score = 0
    # 存所有情感词的位置的列表：
    senLoc = list(senWord.keys())
    #所有否定词的位置列表：
    notLoc = list(notWord.keys())

    senloc = -1  #位置序号，初始为-1，句子中第一个词汇位置是0

    
    # 遍历句中单词列表segResult，i为单词绝对位置
    for i in range(0, len(segResult)):
        # 如果该词为情感词
        if i in senLoc: #说明i位置上是一个情感词
            # loc为情感词位置列表的序号
            senloc += 1
            # 直接添加该情感词分数
            score += W * float(senWord[i])

            if senloc < len(senLoc) - 1:
                # 判断该情感词与下一情感词之间是否有否定词或程度副词
                # j为绝对位置
                for j in range(senLoc[senloc], senLoc[senloc + 1]):
                    # 如果有否定词
                    if j in notLoc:
                        W *= -1
                    #如果有多个否定词，会循环多次
                    # 忽略程度副词

        # i定位至下一个情感词
        if senloc < len(senLoc) - 1:
            i = senLoc[senloc + 1]
    return score



"""
main
"""

#import numpy as np
#import xlwt
import xlrd
#import xlwt

#打开xls文件并读写
from xlutils.copy import copy
rb = xlrd.open_workbook('../data/comments.xls')
wb = copy(rb) 
wsheet = wb.get_sheet(0)
rsheet = rb.sheets()[0]


    
#r1=0 
r2=0 
#r3=0
#r4=0
#r5=0
all=10300  #评论序号0-10300
for i in range(0,all):
    sentence = (rsheet.cell(i, 1).value)    #读取评论
    ans=(rsheet.cell(i, 2).value)  #标准分数
    if(isinstance(sentence, float)):  #处理特殊评论，999,666等
        if(sentence%9==0):
            score=10
        else:
            score=0
    else:
        sentence = "".join(sentence.split())
        wordDict=sent2word(sentence)  #分词
 

        sen=defaultdict() # 句子从list转换成dict格式
        j=0    
        for s in wordDict:
            sen[s]=j
            j=j+1    
        #sen:map(词汇，序号（位置）)

        (senWord, notWord) = classifyWords(sen)#找出情感词位置和否定词位置
        
        score = scoreSent(senWord, notWord, wordDict)  #计算得分
    wsheet.write(i, 5, score)      #写回文件
    wb.save('../result/result.xls')
    
    """
    if score > 1:
        if ans==1:
            r1=r1+1
    elif score < -1:
        if ans==-1:
            r1=r1+1
    elif ans==0:
        r1=r1+1
    
    """
    #以-2，+2为界限，划分-1,0，+1分
    if score > 2:
        if ans==1:
            r2=r2+1
    elif score < -2:
        if ans==-1:
            r2=r2+1
    elif ans==0:
        r2=r2+1
    
    """
    if score > 3:
        if ans==1:
            r3=r3+1
    elif score < -3:
        if ans==-1:
            r3=r3+1
    elif ans==0:
        r3=r3+1
    """
     
    """
    if score > 4:
        if ans == 1:
            r4=r4+1        
    elif score < -4:
        if ans==-1:
            r4=r4+1
    elif ans==0:
            r4=r4+1
    """
            
ratio2=r2*1.0/all  #计算正确率
#ratio1=r1*1.0/all
#ratio3=r3*1.0/all
#ratio4=r4*1.0/all
#print(ratio1)
#print(ratio2)
#print(ratio3)
print(ratio2)
print("finish")
#ratio4: 0.502233   
#ratio3:0.5133980582524272
#ratio2:0.5213592233009708
#ratio1:0.5081553398058253
    


0.8
finish
